In [ ]:
5.
- staticmethod, classmethod
- descriptors
- custom containers
- subclasses and MRO
- metaclasses

## staticmethod, classmethod
Methods without `self`

In [68]:
import json

class MessageBox:
    def __init__(self, title_text, body_text, dismiss_button="Okay"):
        self.title = title_text
        self.body = body_text
        self.buttons = [dismiss_button]

    def __repr__(self):
        return f'<{self.__class__.__name__} {self.title}>'
    
    @classmethod
    def from_json(cls, jtext):
        if not cls.valid_json(jtext):
            raise ValueError('Invalid JSON for this class')
        return cls(**json.loads(jtext))
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button'}
        return required <= set(data) <= everything

m = MessageBox.from_json('''
{
  "title_text": "Alert",
  "body_text": "Something bad happened"
}''')
m

<MessageBox Alert>

In [69]:
m.buttons

['Okay']

In [70]:
MessageBox.valid_json('42')

False

In [64]:
MessageBox.valid_json('{"title_text":"a","body_text":"b"}')

True

In [71]:
MessageBox.from_json('{"title_text":"hello"}')

ValueError: Invalid JSON for this class

In [72]:
class ConfirmBox(MessageBox):
    def __init__(self, title_text, body_text,
                 dismiss_button='Cancel', confirm_button='Proceed'):
        super().__init__(title_text, body_text, dismiss_button)
        self.buttons.append(confirm_button)
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button', 'confirm_button'}
        return required <= set(data) <= everything

c = ConfirmBox.from_json('''
{
  "title_text": "Danger",
  "body_text": "Vent radioactive gas?",
  "confirm_button": "Yes I'm sure"
}''')
c

<ConfirmBox Danger>

In [73]:
c.buttons

['Cancel', "Yes I'm sure"]

In [75]:
ConfirmBox.valid_json(
    '{"title_text":"hi","body_text":"how are you?",'
    '"confirm_button":"good","dismiss_button":"bad"}')

True

## Descriptors
The power behind `property`, `classmethod`, `staticmethod` and normal functions becoming methods

In [83]:
class Tripwire:
    def __get__(self, obj, typ=None):
        print('caught access within', typ)
        if obj:
            print('from object', obj)
        return 'boing'

class Building:
    doorway = Tripwire()

b = Building()
b.doorway

caught access within <class '__main__.Building'>
from object <__main__.Building object at 0x7f9a82bc5b00>


'boing'

In [84]:
Building.doorway

caught access within <class '__main__.Building'>


'boing'

In [86]:
def i_could_be_a_method(self):
    print('hello', self)

i_could_be_a_method.__get__

<method-wrapper '__get__' of function object at 0x7f9a82bca620>

In [89]:
m = i_could_be_a_method.__get__('fake')
m

<bound method i_could_be_a_method of 'fake'>

In [90]:
m()

hello fake


In [107]:
from itertools import count, repeat

class TicketDispenser:
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, typ=None):
        if not obj:
            raise AttributeError('dispenser requires object')
        c = obj.__dict__.setdefault(self.name, count(1))
        return next(c)

    def __set__(self, obj, value):
        obj.__dict__[self.name] = count(value)

    def __delete__(self, obj):
        obj.__dict__[self.name] = repeat('out of order')
        
class ServiceCounter:
    support = TicketDispenser('support')

sc1 = ServiceCounter()
sc2 = ServiceCounter()

print(sc1.support)
print(sc1.support)

1
2


In [108]:
print(sc2.support)
sc2.support = 500
print(sc2.support)
print(sc2.support)
print(sc1.support)
print(sc1.support)

1
500
501
3
4


In [109]:
del sc1.support
print(sc1.support)
print(sc1.support)

out of order
out of order


In [110]:
ServiceCounter.support

AttributeError: dispenser requires object

See also https://docs.python.org/3/howto/descriptor.html#descriptor-protocol